In [54]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
import re
import math
import nltk

In [2]:
df=pd.read_csv("/content/Sentiment.csv")

In [3]:
df

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,...,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,...,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,...,yes,67,Positive\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,...,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13871 entries, 0 to 13870
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         13871 non-null  int64  
 1   candidate                  13775 non-null  object 
 2   candidate_confidence       13871 non-null  float64
 3   relevant_yn                13871 non-null  object 
 4   relevant_yn_confidence     13871 non-null  float64
 5   sentiment                  13871 non-null  object 
 6   sentiment_confidence       13871 non-null  float64
 7   subject_matter             13545 non-null  object 
 8   subject_matter_confidence  13871 non-null  float64
 9   candidate_gold             28 non-null     object 
 10  name                       13871 non-null  object 
 11  relevant_yn_gold           32 non-null     object 
 12  retweet_count              13871 non-null  int64  
 13  sentiment_gold             15 non-null     obj

In [6]:
df.head

<bound method NDFrame.head of           id               candidate  candidate_confidence relevant_yn  \
0          1  No candidate mentioned                1.0000         yes   
1          2            Scott Walker                1.0000         yes   
2          3  No candidate mentioned                1.0000         yes   
3          4  No candidate mentioned                1.0000         yes   
4          5            Donald Trump                1.0000         yes   
...      ...                     ...                   ...         ...   
13866  13867  No candidate mentioned                1.0000         yes   
13867  13868           Mike Huckabee                0.9611         yes   
13868  13869                Ted Cruz                1.0000         yes   
13869  13870            Donald Trump                1.0000         yes   
13870  13871                Ted Cruz                0.9242         yes   

       relevant_yn_confidence sentiment  sentiment_confidence  \
0               

In [14]:
x=df[['text','sentiment']]

In [15]:
x

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [16]:
x.drop(df[df['sentiment']=='Neutral'].index,axis=0,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [17]:
x

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [7]:
df.corr()

,id,candidate_confidence,relevant_yn_confidence,sentiment_confidence,subject_matter_confidence,retweet_count,tweet_id
id,1.000000,0.081504,0.088986,0.051172,0.079515,-0.040689,-0.898911
candidate_confidence,0.081504,1.000000,0.932458,0.506682,0.749057,0.020614,-0.083695
relevant_yn_confidence,0.088986,0.932458,1.000000,0.518682,0.792923,0.021756,-0.088960
sentiment_confidence,0.051172,0.506682,0.518682,1.000000,0.431655,0.008324,-0.062428
subject_matter_confidence,0.079515,0.749057,0.792923,0.431655,1.000000,0.030302,-0.077708
retweet_count,-0.040689,0.020614,0.021756,0.008324,0.030302,1.000000,-0.016831
tweet_id,-0.898911,-0.083695,-0.088960,-0.062428,-0.077708,-0.016831,1.000000


In [11]:
df.columns

Index(['id', 'candidate', 'candidate_confidence', 'relevant_yn',
       'relevant_yn_confidence', 'sentiment', 'sentiment_confidence',
       'subject_matter', 'subject_matter_confidence', 'candidate_gold', 'name',
       'relevant_yn_gold', 'retweet_count', 'sentiment_gold',
       'subject_matter_gold', 'text', 'tweet_coord', 'tweet_created',
       'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')

In [12]:
df.isnull().sum()

id                               0
candidate                       96
candidate_confidence             0
relevant_yn                      0
relevant_yn_confidence           0
sentiment                        0
sentiment_confidence             0
subject_matter                 326
subject_matter_confidence        0
candidate_gold               13843
name                             0
relevant_yn_gold             13839
retweet_count                    0
sentiment_gold               13856
subject_matter_gold          13853
text                             0
tweet_coord                  13850
tweet_created                    0
tweet_id                         0
tweet_location                3912
user_timezone                 4403
dtype: int64

In [18]:
x.groupby('sentiment').describe()

text                                                               
          count unique                                                top freq
sentiment                                                                     
Negative   8493   6129  RT @RWSurferGirl: Jeb Bush reminds me of eleva...  159
Positive   2236   1709  RT @RWSurferGirl: I think Cruz and Trump need ...   72

In [19]:
x = x.drop_duplicates()

In [20]:
x.groupby('sentiment').describe()

text                                                               
          count unique                                                top freq
sentiment                                                                     
Negative   6129   6129  RT @warriorwoman91: I liked her and was happy ...    1
Positive   1709   1709  RT @ScottWalker: Didn't catch the full #GOPdeb...    1

In [37]:
def remove_tags(string):
    removelist = ""
    result = re.sub('RT @','',string)
    result = re.sub('http://.*','',result)
    result = re.sub('[#$%&@â€¦:?]','',result) 
    result = result.lower()
    return result
x['text']=x['text'].apply(lambda cw : remove_tags(cw))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [38]:
x

,text,sentiment
1,scottwalker catch full gopdebate last night. s...,Positive
3,robgeorge carly fiorina trending -- hour debat...,Positive
4,danscavino gopdebate w/ realdonaldtrump delive...,Positive
5,"gregabbott_tx tedcruz ""on first day rescind ev...",Positive
6,warriorwoman91 liked happy heard going moderat...,Negative
...,...,...
13845,"the818 let's play ""how fast distance trump."" g...",Negative
13847,mjtbaum god making appearance gopdebate good...,Positive
13857,"watch fox news, they're asshole gopdebates",Negative
13859,"best line gopdebate ""immigration without assim...",Positive


In [39]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [40]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
x['text'] = x['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [41]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
x['text'] = x.text.apply(lemmatize_text)
x

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,text,sentiment
1,scottwalker catch full gopdebate last night. s...,Positive
3,robgeorge carly fiorina trending -- hour debat...,Positive
4,danscavino gopdebate w/ realdonaldtrump delive...,Positive
5,"gregabbott_tx tedcruz ""on first day rescind ev...",Positive
6,warriorwoman91 liked happy heard going moderat...,Negative
...,...,...
13845,"the818 let's play ""how fast distance trump."" g...",Negative
13847,mjtbaum god making appearance gopdebate good...,Positive
13857,"watch fox news, they're asshole gopdebates",Negative
13859,"best line gopdebate ""immigration without assim...",Positive


In [42]:
s = 0.0
for i in x['text']:
    word_list = i.split()
    s = s + len(word_list)
print("Average length of each review : ",s/x.shape[0])
pos = 0
for i in range(x.shape[0]):
    if x.iloc[i]['sentiment'] == 'Positive':
        pos = pos + 1
neg = x.shape[0]-pos
print("Percentage of reviews with positive sentiment is "+str(pos/x.shape[0]*100)+"%")
print("Percentage of reviews with negative sentiment is "+str(neg/x.shape[0]*100)+"%")

Average length of each review :  10.628604235774432
Percentage of reviews with positive sentiment is 21.804031640724673%
Percentage of reviews with negative sentiment is 78.19596835927533%


In [47]:
reviews = x['text'].values
labels = x['sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [48]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels)

In [49]:
vocab_size = 3000
oov_tok = ''
embedding_dim = 100
max_length = 200 
padding_type='post'
trunc_type='post'

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          300000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [50]:
num_epochs = 5
history = model.fit(train_padded, train_labels, epochs=num_epochs, verbose=1, validation_split=0.1)

Epoch 1/5
166/166 [==============================] - 31s 167ms/step - loss: 0.4886 - accuracy: 0.7940 - val_loss: 0.4155 - val_accuracy: 0.8197
Epoch 2/5
166/166 [==============================] - 27s 161ms/step - loss: 0.3254 - accuracy: 0.8599 - val_loss: 0.4173 - val_accuracy: 0.8027
Epoch 3/5
166/166 [==============================] - 27s 162ms/step - loss: 0.2488 - accuracy: 0.8994 - val_loss: 0.4488 - val_accuracy: 0.8044
Epoch 4/5
166/166 [==============================] - 27s 165ms/step - loss: 0.2512 - accuracy: 0.8981 - val_loss: 0.4880 - val_accuracy: 0.8231
Epoch 5/5
166/166 [==============================] - 27s 161ms/step - loss: 0.1794 - accuracy: 0.9287 - val_loss: 0.5055 - val_accuracy: 0.8146


In [55]:
prediction = model.predict(test_padded)
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred_labels))

Accuracy of prediction on test set :  0.8066326530612244


In [56]:
sentence = ['He is a great leader.','He is a terrible leader.']

sequences = tokenizer.texts_to_sequences(sentence)

padded = pad_sequences(sequences, padding='post', maxlen=max_length)

prediction = model.predict(padded)
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
for i in range(len(sentence)):
    print(sentence[i])
    if pred_labels[i] == 1:
        s = 'Positive'
    else:
        s = 'Negative'
    print("Predicted sentiment : ",s)

He is a great leader.
Predicted sentiment :  Positive
He is a terrible leader.
Predicted sentiment :  Negative
